In [5]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.cluster import KMeans
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import log_loss
from sklearn.model_selection import KFold
from sklearn import preprocessing

# Loading the training and test data

In [6]:
df = pd.read_csv("/kaggle/input/input2/train_folds.csv")
df_test = pd.read_csv("/kaggle/input/playground-series-s3e26/test.csv")
df_sub = pd.read_csv("/kaggle/input/playground-series-s3e26/sample_submission.csv")

# Initalizing Encoder

In [7]:
df = df.drop(['id'],axis=1)
df_test = df_test.drop(['id'],axis=1)

useful_features = [c for c in df.columns if c not in ("Status", "kfold")]
numerical_cols = df[useful_features].select_dtypes(include=['int64', 'float64']).columns

for col in numerical_cols:
    df[col] = np.log1p(df[col])
    df_test[col] = np.log1p(df_test[col])


# Initialize LabelEncoder
label_encoder = LabelEncoder()


final_predictions = []
scores = []

# Training the dataset

In [8]:
for fold in range(5):
    xtrain =  df[df.kfold != fold].reset_index(drop=True)
    xvalid = df[df.kfold == fold].reset_index(drop=True)
    xtest = df_test.copy()
    
    object_columns = xtrain.select_dtypes(include=['object']).columns
    for col in object_columns:
        xtrain[col] = label_encoder.fit_transform(xtrain[col])
    
    object_columns1 = xvalid.select_dtypes(include=['object']).columns
    for col in object_columns1:
        xvalid[col] = label_encoder.fit_transform(xvalid[col])
        
    object_columns2 = xtest.select_dtypes(include=['object']).columns
    for col in object_columns2:
        xtest[col] = label_encoder.fit_transform(xtest[col])
    
    
    ytrain = xtrain['Status']
    yvalid = xvalid['Status']
    
    x_train =  xtrain.drop(['Status'], axis=1)
    x_valid =  xvalid.drop(['Status'], axis=1)
    x_train_final =  x_train.drop(['kfold'], axis=1)
    x_valid_final =  x_valid.drop(['kfold'], axis=1)
    
    scaler = preprocessing.StandardScaler()
    x_train_final[numerical_cols] = scaler.fit_transform(x_train_final[numerical_cols])
    x_valid_final[numerical_cols] = scaler.transform(x_valid_final[numerical_cols])
    xtest[numerical_cols] = scaler.transform(xtest[numerical_cols])
    
    
    model = XGBClassifier(objective='multi:softmax', random_state=42)
    model.fit(x_train_final, ytrain)
    y_pred_proba = model.predict_proba(x_valid_final)
    y_probs = model.predict_proba(xtest)
    final_predictions.append(y_probs)
    loss = log_loss(yvalid, y_pred_proba)
    print(fold, 'Loss:', loss)

preds = np.mean(final_predictions, axis=0)
df_sub['Status_C'] = preds[:, 0]
df_sub['Status_CL'] = preds[:, 1]
df_sub['Status_D'] = preds[:, 2]

df_sub.to_csv('submission.csv',index=False) 

0 Loss: 0.5148158620358305
1 Loss: 0.5170369093648512
2 Loss: 0.5006851268082193
3 Loss: 0.49692741484850966
4 Loss: 0.4927354428657046
